<a href="https://colab.research.google.com/github/floflokie/SSL/blob/main/TP_CLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP : apprentissage multimodal


Dans ce TP, nous allons utiliser le modèle d'apprentissage, FashionCLIP, pré-entraîné sur des images ainsi que des descriptions en langage naturel. Plus particulièrement, nous allons considérer deux cas d'usage :

*   **Moteur de recherche d'images :** il s'agit de trouver, à partir d'une requête en langage naturel, l'image correspondante.

*   **Classification zero-shot :** il s'agit simplement de construire un classifieur d'images (faire correspondre un label à une image).



## Dataset

Nous allons dans un premier temps télécharger les données. Celles-ci provienennt de [Kaggle](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations).

In [ ]:
%%capture
!pip install gdown
!gdown "1igAuIEW_4h_51BG1o05WS0Q0-Cp17_-t&confirm=t"
!unzip data

### Modèle FashionCLIP

Nous allons également télécharger le modèle pré-entraîné.

In [ ]:
%%capture
!pip install -U fashion-clip

In [ ]:
import sys
#sys.path.append("fashion-clip/")
from fashion_clip.fashion_clip import FashionCLIP
import pandas as pd
import numpy as np
from collections import Counter
from PIL import Image
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression

In [ ]:
%%capture
fclip = FashionCLIP('fashion-clip')

FashionCLIP, à l'instar de CLIP, crée un espace vectoriel partagé pour les images et le texte. Cela permet de nombreuses applications, telles que la recherche (trouver l'image la plus similaire à une requête donnée) ou la classification zero-shot.

Il y a principalement deux composants : un encodeur d'image (pour générer un vecteur à partir d'une image) et un encodeur de texte (pour générer un vecteur à partir d'un texte).










<img src="https://miro.medium.com/v2/resize:fit:1400/0*FLNMtW6jK51fm7Og"  width="400">



Nous allons télécharger les données que nous allons ensuite nettoyer.

In [ ]:
articles = pd.read_csv("data_for_fashion_clip/articles.csv")

# Supprimer les éléments ayant la même description
subset = articles.drop_duplicates("detail_desc").copy()

# upprimer les images dont la catégrie n'est pas renseignée
subset = subset[~subset["product_group_name"].isin(["Unknown"])]

# Garder seulement les descriptions dont la longueur est inférieure à 40 tokens
subset = subset[subset["detail_desc"].apply(lambda x : 4 < len(str(x).split()) < 40)]

# Supprimer les articles qui ne sont pas suffisamment fréquents dans le jeu de données
most_frequent_product_types = [k for k, v in dict(Counter(subset["product_type_name"].tolist())).items() if v > 10]
subset = subset[subset["product_type_name"].isin(most_frequent_product_types)]

subset.head(3)

In [ ]:
subset.to_csv("subset_data.csv", index=False)
f"Il y a {len(subset)} éléments dans le dataset"

## Moteur de recherche d'images

Constuire un moteur de recherche qui permet, à partir d'une description en langage naturel, de récupérer l'image correspondante. Mesurer ses performances (précision).

<img src="https://miro.medium.com/v2/resize:fit:1400/1*cnKHgLAumVyuHuK9pkqr7A.gif"  width="800">


In [ ]:
images = ["data_for_fashion_clip/" + str(k) + ".jpg" for k in subset["article_id"].tolist()]
texts = subset["detail_desc"].tolist()

# Créer les représentations vectorielles (embeddings) des images et des descriptions.
batch_size = 32
image_embeddings = fclip.encode_images(images, batch_size=batch_size)
text_embeddings = fclip.encode_text(texts, batch_size=batch_size)


In [ ]:
print(image_embeddings.shape)
print(text_embeddings.shape)

(3104, 512)
(3104, 512)


In [ ]:
def search_engine(query):
  query_embedding = fclip.encode_text([query])[0]
  similarity = np.dot(image_embeddings, query_embedding) / (np.linalg.norm(image_embeddings, axis=1) * np.linalg.norm(query_embedding))
  top_indices = np.argsort(similarity)[::-1][:5]
  return [subset.iloc[i]["article_id"] for i in top_indices]

image_search = search_engine("a pink dress")
print(image_search)
for i in image_search:
  display(Image.open("data_for_fashion_clip/" + str(i) + ".jpg"))

Notre moteur de recherche semble bien fonctionner.

# Classification zero-shot

Construite un classsifieur d'images (prédire le label d'une image). Mesurer ses performances.

<img src="https://miro.medium.com/v2/resize:fit:1400/1*No6ZONpQMIcfFaNMOI5oNw.gif"  width="800">



On utilisera les types de produit comme labels.

In [ ]:
labels

['Belt',
 'Gloves',
 'Other accessories',
 'T-shirt',
 'Socks',
 'Garment Set',
 'Underwear Tights',
 'Bra',
 'Necklace',
 'Skirt',
 'Blouse',
 'Trousers',
 'Dungarees',
 'Top',
 'Cap/peaked',
 'Blazer',
 'Boots',
 'Hair/alice band',
 'Shorts',
 'Coat',
 'Cardigan',
 'Scarf',
 'Sneakers',
 'Ballerinas',
 'Hoodie',
 'Bag',
 'Swimsuit',
 'Hat/brim',
 'Bikini top',
 'Vest top',
 'Dress',
 'Earring',
 'Polo shirt',
 'Sandals',
 'Jumpsuit/Playsuit',
 'Leggings/Tights',
 'Jacket',
 'Shirt',
 'Hat/beanie',
 'Underwear bottom',
 'Swimwear bottom',
 'Other shoe',
 'Bodysuit',
 'Pyjama set',
 'Sweater',
 'Sunglasses']

On obtient de très bon résultats. On a un score de 0 sur top mais c'est un mot peu précis.

Ci-dessous une démonstration du classifier.